In [12]:
import pandas as pd

df = pd.read_csv('human_study_data.csv')

In [13]:
df

,timestamp,prolific_id,age,gender,religion,ethnic_group,education,primary_language,type_of_residence,income,...,social_tendency,hobby,num_places_lived,social_relationship_values,primary_goal,meaningful_events,values_in_friends,what_to_do_with_100_dollars,social_feed_1,social_feed_2
0,3/4/2025 23:14:27,6604cc1105cf0f2779ae1ae8,25-34,Female,Islam,White or Caucasian,Graduate degree (MA/MSc/MPhil/other),English,Suburban,"$20,000 - $29,999",...,Friendly,Photography,3,"Loyalty, Emotional connection",Career growth,Graduating,thoughtful,Pay debt,like-post: post-9e09d3\nshare-post: post-3fa5e...,like-post: post-ddbcb2 \nshare-post: post-c84e...
1,3/4/2025 23:14:59,57d3280454aba00001d9f0f7,25-34,Female,Spiritual but not religious,White or Caucasian,Undergraduate degree (BA/BSc/other),English,Suburban,"$70,000 - $79,999",...,"Friendly, Reserved, Awkward",Video games,5,"Trust, Fun, Loyalty, Intellectual connection, ...",Career growth,Moving from home state,Intellectual,Food,"like-post: post-a02de3, like-post: post-6be7d0...","like-post: post-ddbcb2, like-post: post-c890f0..."
2,3/4/2025 23:16:36,67acfb68c3860389b0c4db4b,25-34,Female,Christianity,White or Caucasian,Undergraduate degree (BA/BSc/other),English,Suburban,"$50,000 - $59,999",...,Friendly,Reading,2,"Trust, Loyalty, Emotional connection",Family,Becoming a mother.,"Mutual care for eachother, understanding of li...",Go on a childless date with my husband.,Like post: post-3b6b25 \nShare post: post-b50e43,Like: post-ddbcb2\nShare: post-29054b
3,3/4/2025 23:29:59,66661b49621ea96feaf4f6f3,35-44,Female,Christianity,Asian,High school diploma/A-levels,English,Suburban,"$50,000 - $59,999",...,Reserved,Being a mom,5,"Trust, Loyalty",Family,Giving birth to first born,Everlasting friendship,Spend it on my son,Some of the post are ridiculous and shouldn’t ...,Would only want fact checked
4,3/4/2025 23:30:35,6789317cf0fc05fe026e0010,25-34,Female,Christianity,Asian,Undergraduate degree (BA/BSc/other),English,Suburban,"$10,000 - $19,999",...,Reserved,cooking,5,"Trust, Fun, Loyalty, Intellectual connection, ...",Family,Giving birth,Trustworthy,Save it,like-post:post-3fa5ef\nlike-post:post-a02de3,like-post:st-9b758b
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199,3/5/2025 2:02:29,67775a3990c24dfe448b78bd,55-64,Male,No Religion,Asian,Graduate degree (MA/MSc/MPhil/other),English,Suburban,"$100,000 - $149,999",...,Reserved,music,5,"Trust, Emotional connection",Financial stability,birth of child,support,save and share,like-post: post-a41a79,like-post: post-bd8ac4
200,3/5/2025 2:11:55,677b706794388e138823dffd,35-44,Male,Christianity,Black or African American,Undergraduate degree (BA/BSc/other),English,Urban,"$90,000 - $99,999",...,"Confident, Friendly, Reserved",football,2,"Fun, Loyalty, Intellectual connection",Family,Experiencing the delivery of my two children,honesty and integrity,"I would buy a gift for my son , his birthday i...","""action"" : likes 1000\n""target"" : post_id: pos...","\n""action"" : likes 10\n""target"" : post_id: pos..."
201,3/5/2025 2:13:56,5d0a8a22c0fd80001626e38d,25-34,Male,Christianity,White or Caucasian,Undergraduate degree (BA/BSc/other),English,Suburban,"$30,000 - $39,999",...,"Reserved, Awkward",Board gaming,3,"Trust, Emotional connection",Financial stability,I lost my mother last year after 5 years of he...,An openness to new opinions and experiences,I'd probably contribute most of it to rebuildi...,like: post-c31764\nlike: post-a41a79\ncomment:...,like: post-1718ae\nadd-note: post-29054b: This...
202,3/5/2025 2:32:22,6792b0c31ed2afe68cdd6ace,35-44,Male,Christianity,White or Caucasian,Graduate degree (MA/MSc/MPhil/other),English,Urban,"$100,000 - $149,999",...,Reserved,watching movir,1,"Loyalty, Intellectual connection",Financial stability,family reunion,loyalty,buy some treats and share fore the people who...,"I’d like the tech/product posts, add a fact-ch...",I would interact with the feed by liking relev...


In [20]:
# drop timestamp column
df = df.drop(columns=['timestamp'])
# convert this df to jsonl
# Save this df to jsonl
df.to_json('human_study_data.jsonl', orient='records', lines=True)


In [ ]:
import json
from openai import OpenAI

# Function to process feed content through GPT-4
def process_feed_content(feed_content, client):
    try:
        completion = client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {"role": "system", "content": "You are a helpful assistant that reformats social media feed actions into JSON format."},
                {
                    "role": "user",
                    "content": f"Please reformat this social media feed content into a JSON object with actions array: {feed_content}"
                }
            ]
        )
        return completion.choices[0].message.content
    except Exception as e:
        print(f"Error processing content: {e}")
        return None

# Initialize OpenAI client
client = OpenAI()

# Create list to store results
results = []

# Process each row
for index, row in df.iterrows():
    try:
        # Extract relevant attributes
        result = {
            "prolific_id": row["prolific_id"],
            "age": row["age"],
            "gender": row["gender"],
            "education": row["education"],
            "social_tendency": row["social_tendency"],
            "primary_goal": row["primary_goal"],
            
        }
        
        # Process social_feed_1 content
        if pd.notna(row["social_feed_1"]):
            formatted_feed = process_feed_content(row["social_feed_1"], client)
            if formatted_feed:
                result["formatted_feed"] = formatted_feed
        
        results.append(result)
        print(result)
        
    except Exception as e:
        print(f"Error processing row {index}: {e}")
        continue

# Save to JSONL file
import json

with open('formatted_social_feeds.jsonl', 'w') as f:
    for result in results:
        f.write(json.dumps(result) + '\n')

print("Processing complete! Results saved to formatted_social_feeds.jsonl")

KeyboardInterrupt: 